# 50 persons
## electrodes : 8, 16, 32, 64

In [23]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [24]:
no_of_patients=50

In [25]:
train=sorted(glob('files/*.edf'))
train=train[:no_of_patients]
train

['files/S001R03.edf',
 'files/S002R03.edf',
 'files/S003R03.edf',
 'files/S004R03.edf',
 'files/S005R03.edf',
 'files/S006R03.edf',
 'files/S007R03.edf',
 'files/S008R03.edf',
 'files/S009R03.edf',
 'files/S010R03.edf',
 'files/S011R03.edf',
 'files/S012R03.edf',
 'files/S013R03.edf',
 'files/S014R03.edf',
 'files/S015R03.edf',
 'files/S016R03.edf',
 'files/S017R03.edf',
 'files/S018R03.edf',
 'files/S019R03.edf',
 'files/S020R03.edf',
 'files/S021R03.edf',
 'files/S022R03.edf',
 'files/S023R03.edf',
 'files/S024R03.edf',
 'files/S025R03.edf',
 'files/S026R03.edf',
 'files/S027R03.edf',
 'files/S028R03.edf',
 'files/S029R03.edf',
 'files/S030R03.edf',
 'files/S031R03.edf',
 'files/S032R03.edf',
 'files/S033R03.edf',
 'files/S034R03.edf',
 'files/S035R03.edf',
 'files/S036R03.edf',
 'files/S037R03.edf',
 'files/S038R03.edf',
 'files/S039R03.edf',
 'files/S040R03.edf',
 'files/S041R03.edf',
 'files/S042R03.edf',
 'files/S043R03.edf',
 'files/S044R03.edf',
 'files/S045R03.edf',
 'files/S0

In [26]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample()
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [27]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [28]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [29]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [30]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(600000, 74950, 600000, 74950)

In [31]:
print(xtest.iloc[:,-1].values)

[-5.7e-05 -4.5e-05 -2.9e-05 ...  9.0e-06  4.0e-06  1.6e-05]


In [32]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.000025,0.000034,0.000019,0.000012,0.000017,0.000013,0.000018,0.000029,0.000014,0.000010,...,0.000020,0.000008,-0.000002,0.000019,0.000013,0.000005,0.000013,0.000007,0.000006,-0.000060
599996,0.000061,0.000068,0.000057,0.000048,0.000050,0.000044,0.000043,0.000048,0.000049,0.000043,...,0.000013,0.000010,-0.000007,0.000011,0.000012,0.000003,0.000013,0.000020,0.000016,-0.000027
599997,0.000066,0.000065,0.000059,0.000050,0.000051,0.000051,0.000054,0.000031,0.000040,0.000035,...,-0.000002,-0.000025,-0.000032,-0.000006,-0.000002,-0.000013,-0.000019,-0.000010,-0.000002,-0.000059
599998,0.000081,0.000087,0.000066,0.000059,0.000069,0.000060,0.000064,0.000054,0.000043,0.000041,...,0.000011,0.000000,-0.000006,0.000014,0.000014,-0.000003,0.000005,0.000000,0.000005,-0.000064


In [33]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [34]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [35]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_1204/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_1204/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_1204/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.000025,0.000034,0.000019,0.000012,0.000017,0.000013,0.000018,0.000029,0.000014,0.000010,...,0.000020,0.000008,-0.000002,0.000019,0.000013,0.000005,0.000013,0.000007,0.000006,-0.000060
599996,0.000061,0.000068,0.000057,0.000048,0.000050,0.000044,0.000043,0.000048,0.000049,0.000043,...,0.000013,0.000010,-0.000007,0.000011,0.000012,0.000003,0.000013,0.000020,0.000016,-0.000027
599997,0.000066,0.000065,0.000059,0.000050,0.000051,0.000051,0.000054,0.000031,0.000040,0.000035,...,-0.000002,-0.000025,-0.000032,-0.000006,-0.000002,-0.000013,-0.000019,-0.000010,-0.000002,-0.000059
599998,0.000081,0.000087,0.000066,0.000059,0.000069,0.000060,0.000064,0.000054,0.000043,0.000041,...,0.000011,0.000000,-0.000006,0.000014,0.000014,-0.000003,0.000005,0.000000,0.000005,-0.000064


In [36]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [37]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [38]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=20,validation_data=(xv8,yv8)
)

Epoch 1/20


18750/18750 [==============================] - 23s 1ms/step - loss: 2.0915 - val_loss: 2.2535
Epoch 2/20
18750/18750 [==============================] - 22s 1ms/step - loss: 1.6643 - val_loss: 2.2405
Epoch 3/20
18750/18750 [==============================] - 22s 1ms/step - loss: 1.5790 - val_loss: 2.2791
Epoch 4/20
18750/18750 [==============================] - 22s 1ms/step - loss: 1.5303 - val_loss: 2.3559
Epoch 5/20
18750/18750 [==============================] - 22s 1ms/step - loss: 1.4971 - val_loss: 2.3528
Epoch 6/20
18750/18750 [==============================] - 22s 1ms/step - loss: 1.4728 - val_loss: 2.3463
Epoch 7/20
18750/18750 [==============================] - 22s 1ms/step - loss: 1.4543 - val_loss: 2.3879
Epoch 8/20
18750/18750 [==============================] - 22s 1ms/step - loss: 1.4373 - val_loss: 2.4360
Epoch 9/20
18750/18750 [==============================] - 22s 1ms/step - loss: 1.4258 - val_loss: 2.4888
Epoch 10/20
18750/18750 [==============================] - 22s 1ms

In [39]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

 211/2343 [=>............................] - ETA: 1s

2343/2343 [==============================] - 2s 707us/step
              precision    recall  f1-score   support

           0       0.22      0.08      0.11      1499
           1       0.22      0.14      0.17      1499
           2       0.25      0.45      0.32      1499
           3       0.54      0.49      0.51      1499
           4       0.70      0.65      0.68      1499
           5       0.43      0.52      0.47      1499
           6       0.45      0.62      0.52      1499
           7       0.64      0.39      0.49      1499
           8       0.52      0.60      0.55      1499
           9       0.53      0.32      0.40      1499
          10       0.27      0.28      0.28      1499
          11       0.25      0.21      0.23      1499
          12       0.27      0.27      0.27      1499
          13       0.18      0.23      0.20      1499
          14       0.17      0.23      0.20      1499
          15       0.49      0.35      0.41      1499
          16       0.1

## 0-16

In [40]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [41]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_1204/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_1204/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_1204/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.000025,0.000034,0.000019,0.000012,0.000017,0.000013,0.000018,0.000029,0.000014,0.000010,...,0.000020,0.000008,-0.000002,0.000019,0.000013,0.000005,0.000013,0.000007,0.000006,-0.000060
599996,0.000061,0.000068,0.000057,0.000048,0.000050,0.000044,0.000043,0.000048,0.000049,0.000043,...,0.000013,0.000010,-0.000007,0.000011,0.000012,0.000003,0.000013,0.000020,0.000016,-0.000027
599997,0.000066,0.000065,0.000059,0.000050,0.000051,0.000051,0.000054,0.000031,0.000040,0.000035,...,-0.000002,-0.000025,-0.000032,-0.000006,-0.000002,-0.000013,-0.000019,-0.000010,-0.000002,-0.000059
599998,0.000081,0.000087,0.000066,0.000059,0.000069,0.000060,0.000064,0.000054,0.000043,0.000041,...,0.000011,0.000000,-0.000006,0.000014,0.000014,-0.000003,0.000005,0.000000,0.000005,-0.000064


In [42]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [46]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [47]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=20,validation_data=(xv16,yv16)
)

Epoch 1/20


18750/18750 [==============================] - 26s 1ms/step - loss: 1.4957 - val_loss: 1.9843
Epoch 2/20
18750/18750 [==============================] - 24s 1ms/step - loss: 0.8323 - val_loss: 1.9573
Epoch 3/20
18750/18750 [==============================] - 25s 1ms/step - loss: 0.7108 - val_loss: 2.1479
Epoch 4/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.6497 - val_loss: 2.1520
Epoch 5/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.6083 - val_loss: 2.1886
Epoch 6/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.5785 - val_loss: 2.1889
Epoch 7/20
18750/18750 [==============================] - 22s 1ms/step - loss: 0.5564 - val_loss: 2.3994
Epoch 8/20
18750/18750 [==============================] - 22s 1ms/step - loss: 0.5390 - val_loss: 2.4482
Epoch 9/20
18750/18750 [==============================] - 22s 1ms/step - loss: 0.5242 - val_loss: 2.3509
Epoch 10/20
18750/18750 [==============================] - 22s 1ms

In [48]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

 362/2343 [===>..........................] - ETA: 1s

2343/2343 [==============================] - 2s 701us/step
              precision    recall  f1-score   support

           0       0.49      0.40      0.44      1499
           1       0.43      0.35      0.39      1499
           2       0.28      0.43      0.34      1499
           3       0.51      0.47      0.49      1499
           4       0.90      0.83      0.86      1499
           5       0.65      0.52      0.58      1499
           6       0.72      0.84      0.78      1499
           7       0.90      0.52      0.66      1499
           8       0.45      0.80      0.57      1499
           9       0.63      0.67      0.65      1499
          10       0.40      0.23      0.30      1499
          11       0.61      0.46      0.52      1499
          12       0.41      0.62      0.50      1499
          13       0.27      0.42      0.33      1499
          14       0.26      0.43      0.32      1499
          15       0.57      0.10      0.18      1499
          16       0.3

## 0-32

In [49]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [50]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_1204/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_1204/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_1204/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [51]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [52]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [53]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=20,validation_data=(xv32,yv32)
)

Epoch 1/20


18750/18750 [==============================] - 23s 1ms/step - loss: 1.0213 - val_loss: 1.6089
Epoch 2/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.3814 - val_loss: 1.5283
Epoch 3/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.2710 - val_loss: 1.5620
Epoch 4/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.2236 - val_loss: 1.4365
Epoch 5/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1937 - val_loss: 1.4429
Epoch 6/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1738 - val_loss: 1.5108
Epoch 7/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1593 - val_loss: 1.5520
Epoch 8/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1472 - val_loss: 1.4733
Epoch 9/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1385 - val_loss: 1.6842
Epoch 10/20
18750/18750 [==============================] - 23s 1ms

In [54]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

 591/2343 [======>.......................] - ETA: 1s

2343/2343 [==============================] - 2s 688us/step
              precision    recall  f1-score   support

           0       0.79      0.85      0.82      1499
           1       0.74      0.80      0.77      1499
           2       0.70      0.74      0.72      1499
           3       0.66      0.52      0.58      1499
           4       0.90      0.96      0.93      1499
           5       0.91      0.96      0.94      1499
           6       0.94      0.96      0.95      1499
           7       0.96      0.77      0.86      1499
           8       0.43      0.85      0.57      1499
           9       0.81      0.84      0.82      1499
          10       0.91      0.90      0.90      1499
          11       0.96      0.97      0.97      1499
          12       0.78      0.83      0.80      1499
          13       0.67      0.90      0.77      1499
          14       0.60      0.82      0.69      1499
          15       0.85      0.19      0.31      1499
          16       0.5

## 0-64

In [55]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [56]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [57]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [58]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=20,validation_data=(xv,yv)
)

Epoch 1/20


18750/18750 [==============================] - 24s 1ms/step - loss: 0.7995 - val_loss: 1.6315
Epoch 2/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.2948 - val_loss: 1.6663
Epoch 3/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.2131 - val_loss: 1.8182
Epoch 4/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1737 - val_loss: 1.6373
Epoch 5/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1490 - val_loss: 1.6532
Epoch 6/20
18750/18750 [==============================] - 24s 1ms/step - loss: 0.1307 - val_loss: 1.8135
Epoch 7/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1169 - val_loss: 1.7905
Epoch 8/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1073 - val_loss: 1.8474
Epoch 9/20
18750/18750 [==============================] - 23s 1ms/step - loss: 0.0990 - val_loss: 1.7226
Epoch 10/20
18750/18750 [==============================] - 23s 1ms

In [59]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

2343/2343 [==============================] - 2s 701us/step
              precision    recall  f1-score   support

           0       0.81      0.88      0.85      1499
           1       0.91      0.89      0.90      1499
           2       0.73      0.73      0.73      1499
           3       0.92      0.94      0.93      1499
           4       0.93      0.99      0.96      1499
           5       0.96      0.97      0.96      1499
           6       0.94      0.99      0.96      1499
           7       0.97      0.85      0.90      1499
           8       0.53      0.87      0.66      1499
           9       0.71      0.87      0.78      1499
          10       0.90      0.83      0.86      1499
          11       0.98      0.90      0.94      1499
          12       0.61      0.92      0.74      1499
          13       0.74      0.92      0.82      1499
          14       0.67      0.87      0.76      1499
          15       0.89      0.63      0.74      1499
          16       0.5